In [ ]:
# ✅ Final Streamlit Dashboard for CustomerSegmentAI360 (Auto-Preprocessing + Correlation & Elbow)

import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

st.set_page_config(page_title="360° AI Customer Dashboard", layout="wide")

# ------------------------
# 🔐 1. Admin Login
# ------------------------
if "authenticated" not in st.session_state:
    st.session_state["authenticated"] = False

if not st.session_state.authenticated:
    st.title("🔐 Admin Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    if st.button("Login"):
        if username == "admin" and password == "pass123":
            st.session_state.authenticated = True
            st.rerun()
        else:
            st.error("Invalid credentials.")
    st.stop()

# ------------------------
# 📥 2. Load and Preprocess Data
# ------------------------
@st.cache_data

def load_default():
    return pd.read_csv("rfm_final_with_predictions.csv")

def preprocess_and_segment(df):
    df = df.drop_duplicates().dropna()
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

    if {'recency', 'frequency', 'monetary'}.issubset(df.columns):
        model_features = ['recency', 'frequency', 'monetary']
        try:
            model = joblib.load("segment_model_xgboost.pkl")
            df['Segment_Pred'] = model.predict(df[model_features])
            segment_map = {0: "Window Shopper", 1: "Bargain Hunter", 2: "Loyalist", 3: "VIP"}
            df['Segment_Label'] = df['Segment_Pred'].map(segment_map)
        except Exception as e:
            st.warning(f"Segmentation model error: {e}")
    return df

data_file = st.sidebar.file_uploader("Upload Your CSV File", type=["csv"])
if data_file is not None:
    data = pd.read_csv(data_file)
    data = preprocess_and_segment(data)
    st.sidebar.success("✅ File uploaded and processed")
else:
    data = load_default()
    st.sidebar.info("Using default dataset")

# ------------------------
# 🧾 3. Segment Name Mapping
# ------------------------
segment_name_map = {
    0: "Window Shopper",
    1: "Bargain Hunter",
    2: "Loyalist",
    3: "VIP"
}
if 'Segment_Pred' in data.columns:
    data['Segment_Label'] = data['Segment_Pred'].map(segment_name_map)

# ------------------------
# 🎛️ 4. Filter Sidebar
# ------------------------
st.sidebar.header("🔍 Filter Customers")
filterable_cols = [col for col in data.columns if data[col].nunique() < 25 and data[col].dtype in ["int64", "object"]]
for col in filterable_cols:
    unique_vals = data[col].dropna().unique().tolist()
    selected_vals = st.sidebar.multiselect(f"{col}", unique_vals, default=unique_vals)
    data = data[data[col].isin(selected_vals)]

# ------------------------
# 📊 5. Dashboard Tabs
# ------------------------
tabs = st.tabs(["📋 Overview", "🧠 Predictions", "📈 Visuals", "📌 Segment Insights"])

# ------------------------
# 📋 Overview
# ------------------------
with tabs[0]:
    st.title("📊 Customer Overview")
    k1, k2, k3 = st.columns(3)
    k1.metric("Total Customers", f"{len(data):,}")
    if 'Churn_Pred' in data.columns:
        k2.metric("Avg Churn Probability", f"{data['Churn_Pred'].mean():.2f}")
    if 'CLV' in data.columns:
        k3.metric("Avg CLV", f"${data['CLV'].mean():,.2f}")
    st.dataframe(data[['CustomerID', 'CLV', 'Churn_Pred', 'Segment_Label']].head(10) if 'CLV' in data.columns and 'Churn_Pred' in data.columns else data.head(10))

# ------------------------
# 🧠 Predictions
# ------------------------
with tabs[1]:
    st.subheader("🔎 Lookup Customer Predictions")
    lookup = st.text_input("Enter Customer ID")
    if lookup:
        if 'CustomerID' in data.columns:
            user_row = data[data['CustomerID'].astype(str) == lookup]
            if not user_row.empty:
                st.dataframe(user_row.T)
                prob = user_row.iloc[0].get('Churn_Pred', None)
                clv = user_row.iloc[0].get('CLV', None)
                seg = user_row.iloc[0].get('Segment_Label', 'Unknown')
                st.info(f"🧠 Segment: {seg}")
                if prob is not None and prob > 0.8:
                    st.warning("⚠️ High Churn Risk")
                elif clv is not None and clv > 200:
                    st.success("💎 VIP Customer")
                else:
                    st.success("🙂 Low Risk")
            else:
                st.error("Customer ID not found")
        else:
            st.warning("Customer ID column not found in dataset.")

# ------------------------
# 📈 Visuals
# ------------------------
with tabs[2]:
    st.subheader("📈 Visual Analytics")
    col1, col2 = st.columns(2)

    if 'CLV' in data.columns:
        fig1 = px.histogram(data, x='CLV', nbins=50, title="CLV Distribution")
        col1.plotly_chart(fig1, use_container_width=True)

    if 'Churn_Pred' in data.columns:
        fig2 = px.histogram(data, x='Churn_Pred', nbins=50, title="Churn Probability Distribution")
        col2.plotly_chart(fig2, use_container_width=True)

    if 'Segment_Label' in data.columns and 'CLV' in data.columns:
        fig3 = px.box(data, x='Segment_Label', y='CLV', color='Segment_Label', title="CLV by Segment")
        st.plotly_chart(fig3, use_container_width=True)

    if 'Segment_Label' in data.columns and 'Churn_Pred' in data.columns:
        fig4 = px.box(data, x='Segment_Label', y='Churn_Pred', color='Segment_Label', title="Churn Probability by Segment")
        st.plotly_chart(fig4, use_container_width=True)

    if 'CLV' in data.columns:
        fig5 = px.line(data.sort_values(by='CLV', ascending=False).reset_index(), y='CLV', title="Predicted CLV Line Chart")
        st.plotly_chart(fig5, use_container_width=True)

    # Correlation Heatmap
    st.subheader("📊 Correlation Matrix")
    numeric_cols = data.select_dtypes(include=np.number)
    if not numeric_cols.empty:
        corr = numeric_cols.corr()
        fig_corr = px.imshow(corr, text_auto=True, aspect="auto", title="Feature Correlation Heatmap")
        st.plotly_chart(fig_corr, use_container_width=True)

    # Elbow Method
    st.subheader("📍 Elbow Method for KMeans")
    if {'recency', 'frequency', 'monetary'}.issubset(data.columns):
        scaled = StandardScaler().fit_transform(data[['recency', 'frequency', 'monetary']])
        distortions = []
        K = range(1, 10)
        for k in K:
            kmeans = KMeans(n_clusters=k, random_state=42)
            kmeans.fit(scaled)
            distortions.append(kmeans.inertia_)
        fig_elbow, ax = plt.subplots()
        ax.plot(K, distortions, 'bo-')
        ax.set_xlabel('Number of clusters')
        ax.set_ylabel('Inertia')
        ax.set_title('Elbow Method For Optimal k')
        st.pyplot(fig_elbow)

# ------------------------
# 📌 Segment Insights
# ------------------------
with tabs[3]:
    st.subheader("📌 Actionable Segment Insights")
    if 'Segment_Label' in data.columns:
        for seg in data['Segment_Label'].unique():
            st.markdown(f"### 🎯 {seg}")
            if "vip" in str(seg).lower():
                st.markdown("- Offer exclusive deals and early access to new products.")
            elif "window" in str(seg).lower():
                st.markdown("- Retarget with ads and email nudges to convert interest to purchase.")
            elif "bargain" in str(seg).lower():
                st.markdown("- Highlight discounts and promote value bundles.")
            elif "loyal" in str(seg).lower():
                st.markdown("- Reward with loyalty perks and referrals.")
            else:
                st.markdown("- Analyze behavior and personalize communication.")
            subset = data[data['Segment_Label'] == seg]
            st.write(f"Total customers in this segment: {len(subset)}")
            st.dataframe(subset[['CustomerID', 'CLV', 'Churn_Pred']].head(5) if 'CLV' in data.columns and 'Churn_Pred' in data.columns else subset.head(5))

# ------------------------
# 📤 Export Options
# ------------------------
st.markdown("---")
st.subheader("📤 Export Segments")
vips = data[data['Segment_Label'] == 'VIP'] if 'Segment_Label' in data.columns else pd.DataFrame()
risks = data[data['Churn_Pred'] > 0.7] if 'Churn_Pred' in data.columns else pd.DataFrame()
col_exp1, col_exp2 = st.columns(2)
if not vips.empty:
    col_exp1.download_button("📥 Download VIPs", vips.to_csv(index=False), "vip_customers.csv")
if not risks.empty:
    col_exp2.download_button("📥 Download High-Risk", risks.to_csv(index=False), "high_risk_customers.csv")

# ------------------------
# 📋 Full Table View
# ------------------------
st.subheader("📋 Full Customer Table")
st.dataframe(data.reset_index(drop=True))
